# Driver Drowsiness Detection

In [1]:
import cv2
import tensorflow as tf
import keras
import pygame
from pygame import mixer
import matplotlib.pyplot as plt
import pygame.camera
import numpy as np
from keras.models import load_model
import time
import os

print(f"Version opencv: {cv2.__version__}")
print(f"Version tensorflow: {tf.__version__}")
print(f"Version keras: {keras.__version__}")
print(f"Version pygame: {pygame.__version__}")
print(f"Version numpy: {np.__version__}")

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Version opencv: 4.5.5
Version tensorflow: 2.9.1
Version keras: 2.9.0
Version pygame: 2.1.2
Version numpy: 1.20.3


### Take picture using opencv

In [ ]:
cam = cv2.VideoCapture(0)
result, image = cam.read()

if result:
    # show the image
    plt.imshow("Foto", image)
else:
    print("Move to this part is input image has some error")

### Create sound

In [3]:
mixer.init()
sound = mixer.Sound('Drowsiness detection/alarm.wav')

### Take picture using pygame and select ROI manually

In [ ]:
number_of_pictures = 1

def take_picture_roi(n):
    if n == 0:
        return
    else:
        # initializing the camera
        pygame.camera.init()
        # make the list of all available cameras
        camlist = pygame.camera.list_cameras()
        # if camera is detected or not
        if camlist:
            # initializing the cam variable with default camera
            cam = pygame.camera.Camera(camlist[0], (640, 480))
            # opening the camera
            cam.start()
            # capturing the single image
            image = cam.get_image()
            # saving the image
            pygame.image.save(image, "filename.jpg")
        # if camera is not detected the moving to else part
        else:
            print("No camera on current device")
            
        cam.stop()
         #image_path
        img_path="filename.jpg"
        #read image
        img_raw = cv2.imread(img_path)
        #select ROI function
        roi = cv2.selectROI(img_raw)
        #print rectangle points of selected roi
        print(roi)
        #Crop selected roi from raw image
        roi_cropped = img_raw[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
        #show cropped image
        #cv2.imshow("ROI", roi_cropped)
        cv2.imwrite("crop.jpeg",roi_cropped)
        #hold window
        #cv2.waitKey(0)
        #cv2.destroyWindow()
        n = n - 1
        cv2.destroyAllWindows()
    
if __name__ == "__main__":
    take_picture_roi(number_of_pictures)
    print("All pictures have been taken.")

### Take picture

In [10]:
number_of_pics = 1

def take_picture(n):
    if n == 0:
        return
    else:
        # initializing the camera
        pygame.camera.init()
        # make the list of all available cameras
        camlist = pygame.camera.list_cameras()
        # if camera is detected or not
        if camlist:
            # initializing the cam variable with default camera
            cam = pygame.camera.Camera(camlist[0], (640, 480))
            # opening the camera
            cam.start()
            # capturing the single image
            image = cam.get_image()
            # saving the image
            pygame.image.save(image, "filename.jpg")
        # if camera is not detected the moving to else part
        else:
            print("No camera on current device")
            
        cam.stop()
    
if __name__ == "__main__":
    take_picture(number_of_pics)
    print("All pictures have been taken.")

All pictures have been taken.


### Apply model on live taken pictures

In [14]:
img_path="filename.jpg"
#read image
image_color = cv2.imread(img_path)
height,width = image_color.shape[:2]
gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)

print(type(gray))
print(gray.shape)

face=cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_default.xml")
#face = cv2.CascadeClassifier("/Drowsiness detection/haar cascade files/haarcascade_frontalface_alt.xml")
faces = face.detectMultiScale(gray)

leye = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_lefteye_2splits.xml")
#face = cv2.CascadeClassifier("/Drowsiness detection/haar cascade files/haarcascade_frontalface_alt.xml")
left_eyes = face.detectMultiScale(gray)

reye = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_righteye_2splits.xml")
#face = cv2.CascadeClassifier("/Drowsiness detection/haar cascade files/haarcascade_frontalface_alt.xml")
right_eyes = face.detectMultiScale(gray)

lbl=['Close','Open']
model = load_model('Drowsiness detection/models/cnncat2.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

cv2.rectangle(image_color, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

for (x,y,w,h) in faces: 
        fot01 = cv2.rectangle(image_color, (x,y), (x+w, y+h), (100,100,100), 1 )
        
for (x,y,w,h) in right_eyes:
        r_eye=image_color[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye= r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict(r_eye)
        print(f"rpred: {rpred}")
        rprednew = []
        for i in rpred:
            for j in i:
                if j > 0.51:
                    j = 1
                    rprednew.append(j)
                else:
                    j = 0
                    rprednew.append(j)
        if(rprednew[0]==0):
            lbl='Open'
        if(rprednew[0]==1):
            lbl='Closed'
        break
for (x,y,w,h) in left_eyes:
    l_eye=image_color[y:y+h,x:x+w]
    count=count+1
    l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)
    l_eye = cv2.resize(l_eye,(24,24))
    l_eye= l_eye/255
    l_eye=l_eye.reshape(24,24,-1)
    l_eye = np.expand_dims(l_eye,axis=0)
    lpred = model.predict(l_eye)
    print(f"lpred: {lpred}")
    lprednew = []
    for i in lpred:
        for j in i:
            if j > 0.51:
                j = 1
                lprednew.append(j)
            else:
                j = 0
                lprednew.append(j)
    print(f"lprednew: {lprednew}")
    if (lprednew[0]==0):
        lbl='Open'
    if (lprednew[0]==1):
        print(f"2e element lprednew: {lprednew[1]}")
        lbl='Closed'
    break
if(rprednew[0]==1 and lprednew[0]==1):
    score=score+1
    cv2.putText(image_color,"Closed:"+str(score),(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
# if(rpred[0]==1 or lpred[0]==1):
else:
    score=score-1
    cv2.putText(image_color,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
if(score<0):
    score=0
    cv2.putText(image_color,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
if(score>15):
    #person is feeling sleepy so we beep the alarm
    cv2.imwrite(os.path.join(path,'image.jpg'),image_color)
    try:
        sound.play()
    except: # isplaying = False
        pass
    if(thicc<16):
        thicc= thicc+2
    else:
        thicc=thicc-2
        if(thicc<2):
            thicc=2
cv2.rectangle(image_color,(0,0),(width,height),(0,0,255),thicc)
cv2.imshow('frame',image_color)
if cv2.waitKey(1) & 0xFF == ord('q'):
    cv2.destroyAllWindows()
    #break
#cap.release()


<class 'numpy.ndarray'>
(720, 1280)
1/1 [==============================] - 0s 85ms/step
rpred: [[0.97756165 0.0224383 ]]
1/1 [==============================] - 0s 22ms/step
lpred: [[0.97756165 0.0224383 ]]
lprednew: [1, 0]
2e element lprednew: 0


### Apply model on dataset of faces

In [10]:
images_path="dataset_new/test/testimages"

def model_on_dataset(images_path):
    for subdir, dirs, files in os.walk(images_path):
        for image in files:
            path_file = os.path.join(subdir, image)
            print(f"image:{image}")
            image_color = cv2.imread(path_file)
            height,width = image_color.shape[:2]
            gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
            print(type(gray))
            print(gray.shape)
            face=cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_default.xml")
            #face = cv2.CascadeClassifier("/Drowsiness detection/haar cascade files/haarcascade_frontalface_alt.xml")
            faces = face.detectMultiScale(gray)
            leye = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_lefteye_2splits.xml")
            #face = cv2.CascadeClassifier("/Drowsiness detection/haar cascade files/haarcascade_frontalface_alt.xml")
            left_eyes = face.detectMultiScale(gray)
            reye = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_righteye_2splits.xml")
            #face = cv2.CascadeClassifier("/Drowsiness detection/haar cascade files/haarcascade_frontalface_alt.xml")
            right_eyes = face.detectMultiScale(gray)
            lbl=['Close','Open']
            model = load_model('Drowsiness detection/models/cnncat2.h5')
            path = os.getcwd()
            cap = cv2.VideoCapture(0)
            font = cv2.FONT_HERSHEY_COMPLEX_SMALL
            count=0
            score=0
            thicc=2
            rpred=[99]
            lpred=[99]
            cv2.rectangle(image_color, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )
            for (x,y,w,h) in faces: 
                    fot01 = cv2.rectangle(image_color, (x,y), (x+w, y+h), (100,100,100), 1 )
            for (x,y,w,h) in right_eyes:
                    r_eye=image_color[y:y+h,x:x+w]
                    count=count+1
                    r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
                    r_eye = cv2.resize(r_eye,(24,24))
                    r_eye= r_eye/255
                    r_eye= r_eye.reshape(24,24,-1)
                    r_eye = np.expand_dims(r_eye,axis=0)
                    rpred = model.predict(r_eye)
                    print(f"rpred: {rpred}")
                    rprednew = []
                    for i in rpred:
                        for j in i:
                            if j > 0.51:
                                j = 1
                                rprednew.append(j)
                            else:
                                j = 0
                                rprednew.append(j)
                    if(rprednew[0]==0):
                        lbl='Open'
                    if(rprednew[0]==1):
                        lbl='Closed'
                    break
            for (x,y,w,h) in left_eyes:
                l_eye=image_color[y:y+h,x:x+w]
                count=count+1
                l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)
                l_eye = cv2.resize(l_eye,(24,24))
                l_eye= l_eye/255
                l_eye=l_eye.reshape(24,24,-1)
                l_eye = np.expand_dims(l_eye,axis=0)
                lpred = model.predict(l_eye)
                print(f"lpred: {lpred}")
                lprednew = []
                for i in lpred:
                    for j in i:
                        if j > 0.51:
                            j = 1
                            lprednew.append(j)
                        else:
                            j = 0
                            lprednew.append(j)
                print(f"lprednew: {lprednew}")
                if (lprednew[0]==0):
                    lbl='Open'
                if (lprednew[0]==1):
                    print(f"2e element lprednew: {lprednew[1]}")
                    lbl='Closed'
                break
            if(rprednew[0]==1 and lprednew[0]==1):
                score=score+1
                cv2.putText(image_color,"Closed:"+str(score),(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            # if(rpred[0]==1 or lpred[0]==1):
            else:
                score=score-1
                cv2.putText(image_color,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            if(score<0):
                score=0
                cv2.putText(image_color,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            if(score>15):
                #person is feeling sleepy so we beep the alarm
                cv2.imwrite(os.path.join(path,'image.jpg'),image_color)
                try:
                    sound.play()
                except: # isplaying = False
                    pass
                if(thicc<16):
                    thicc= thicc+2
                else:
                    thicc=thicc-2
                    if(thicc<2):
                        thicc=2
            cv2.rectangle(image_color,(0,0),(width,height),(0,0,255),thicc)
            cv2.imshow('frame',image_color)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyAllWindows()

In [11]:
model_on_dataset(images_path)

image:n9Eaf2lOZ1kFLYH-Pl4G.jpg
<class 'numpy.ndarray'>
(450, 319)
1/1 [==============================] - 0s 79ms/step
rpred: [[1.0000000e+00 2.0389291e-16]]
1/1 [==============================] - 0s 18ms/step
lpred: [[1.0000000e+00 2.0389291e-16]]
lprednew: [1, 0]
2e element lprednew: 0
image:woman-closed-eyes-portrait-young-beautiful-naked-shoulders-posing-white-background-103120013.jpg
<class 'numpy.ndarray'>
(533, 800)
1/1 [==============================] - 0s 69ms/step
rpred: [[1.0000000e+00 1.9246008e-11]]
1/1 [==============================] - 0s 23ms/step
lpred: [[1.0000000e+00 1.9246008e-11]]
lprednew: [1, 0]
2e element lprednew: 0
image:images.jpeg
<class 'numpy.ndarray'>
(253, 199)
1/1 [==============================] - 0s 73ms/step
rpred: [[1.0000000e+00 2.4032846e-12]]
1/1 [==============================] - 0s 22ms/step
lpred: [[1.0000000e+00 2.4032846e-12]]
lprednew: [1, 0]
2e element lprednew: 0
image:1234.jpg
<class 'numpy.ndarray'>
(1316, 858)
1/1 [=====================